#-----------------------------Data Creation-----------------------------#

In [7]:
import pandas as pd
import numpy as np

In [2]:
products = pd.DataFrame({
    "product_id": ["P001", "P002", "P003", "P004", "P005"],
    "product_name": [
        "Lux Soap",
        "GoodDay Biscuits",
        "Colgate Toothpaste",
        "Sunsilk Shampoo",
        "Pepsi Can"
    ],
    "category": [
        "Personal Care",
        "Food",
        "Oral Care",
        "Personal Care",
        "Beverages"
    ],
    "cost_price": [18, 8, 40, 90, 20],
    "selling_price": [25, 12, 55, 120, 30]
})


In [4]:
products

,product_id,product_name,category,cost_price,selling_price
0,P001,Lux Soap,Personal Care,18,25
1,P002,GoodDay Biscuits,Food,8,12
2,P003,Colgate Toothpaste,Oral Care,40,55
3,P004,Sunsilk Shampoo,Personal Care,90,120
4,P005,Pepsi Can,Beverages,20,30


In [5]:
dates = pd.date_range(start="2024-01-01", end="2024-01-31")
warehouses = ["Bangalore", "Mumbai", "Delhi"]

In [9]:
inventory_data = []

for date in dates:
    for product in products["product_id"]:
        for wh in warehouses:
            stock = np.random.randint(0, 2000)  # includes stockouts
            inventory_data.append([date, product, wh, stock])

inventory = pd.DataFrame(
    inventory_data,
    columns=["date", "product_id", "warehouse", "stock_level"]
)

In [11]:
inventory

,date,product_id,warehouse,stock_level
0,2024-01-01,P001,Bangalore,209
1,2024-01-01,P001,Mumbai,1298
2,2024-01-01,P001,Delhi,1020
3,2024-01-01,P002,Bangalore,113
4,2024-01-01,P002,Mumbai,349
...,...,...,...,...
460,2024-01-31,P004,Mumbai,1753
461,2024-01-31,P004,Delhi,1236
462,2024-01-31,P005,Bangalore,1120
463,2024-01-31,P005,Mumbai,1455


In [13]:
order_ids = range(1001, 1201)

orders = pd.DataFrame({
    "order_id": order_ids,
    "product_id": np.random.choice(products["product_id"], size=200),
    "order_date": np.random.choice(dates, size=200),
    "quantity": np.random.randint(50, 500, size=200)
})


In [14]:
orders["delivery_date"] = orders["order_date"] + pd.to_timedelta(
    np.random.randint(1, 7, size=len(orders)), unit="D"
)


In [15]:
orders.head()

,order_id,product_id,order_date,quantity,delivery_date
0,1001,P002,2024-01-03,222,2024-01-07
1,1002,P001,2024-01-20,183,2024-01-25
2,1003,P001,2024-01-22,473,2024-01-26
3,1004,P002,2024-01-16,121,2024-01-22
4,1005,P001,2024-01-25,312,2024-01-26


In [16]:
months = pd.period_range("2024-01", "2024-03", freq="M")

forecast_data = []

for product in products["product_id"]:
    for month in months:
        forecast = np.random.randint(4000, 10000)
        actual = forecast + np.random.randint(-2000, 2000)
        forecast_data.append([product, month, forecast, actual])

forecast = pd.DataFrame(
    forecast_data,
    columns=["product_id", "month", "forecast_qty", "actual_demand"]
)


In [18]:
forecast

,product_id,month,forecast_qty,actual_demand
0,P001,2024-01,5537,4051
1,P001,2024-02,7360,8812
2,P001,2024-03,7904,8494
3,P002,2024-01,8732,9398
4,P002,2024-02,5993,5302
5,P002,2024-03,4201,2537
6,P003,2024-01,9170,7390
7,P003,2024-02,6637,5198
8,P003,2024-03,6685,7101
9,P004,2024-01,8767,7828


#-----------------------------Data Cleaning-----------------------------#

#Handling Missing Values

In [19]:
#Adding missing values
inventory.loc[inventory.sample(frac=0.01).index, "stock_level"] = np.nan

In [21]:
#Checking missing values
inventory.isna().sum()

date           0
product_id     0
warehouse      0
stock_level    5
dtype: int64

In [24]:
#fixing missing stock levels
inventory["stock_level"] = inventory["stock_level"].fillna(0)

#Fix Date Columns

In [25]:
inventory["date"] = pd.to_datetime(inventory["date"])
orders["order_date"] = pd.to_datetime(orders["order_date"])
orders["delivery_date"] = pd.to_datetime(orders["delivery_date"])


#Remove Duplicates

In [28]:
inventory.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)


#Feature Engineering - Lead Time KPI

In [30]:
orders["lead_time_days"] = (
    orders["delivery_date"] - orders["order_date"]
).dt.days

orders[["order_date", "delivery_date", "lead_time_days"]].head()

,order_date,delivery_date,lead_time_days
0,2024-01-03,2024-01-07,4
1,2024-01-20,2024-01-25,5
2,2024-01-22,2024-01-26,4
3,2024-01-16,2024-01-22,6
4,2024-01-25,2024-01-26,1


#Saving Cleaned Data as CSV for EDA,etc.

In [40]:
products.to_csv(r"C:\Users\keert\Desktop\Supply-Chain-Analytics\data\cleaned\products.csv", index=False)
inventory.to_csv(r"C:\Users\keert\Desktop\Supply-Chain-Analytics\data\cleaned\inventory.csv", index=False)
orders.to_csv(r"C:\Users\keert\Desktop\Supply-Chain-Analytics\data\cleaned\orders.csv", index=False)
forecast.to_csv(r"C:\Users\keert\Desktop\Supply-Chain-Analytics\data\cleaned\demand_forecast.csv", index=False)